# Code for simulating C4 biochemical photosynthesis, reading measured photosynthetic reponses to intercecullar CO2 and radiation (A/CI-AI) and deriving posterior distributions for biochemical photosynthetic coefficients Vcmax, Vpmax, ε, Jmax and Rd by fitting individual paired ACI-AI curves

Author: Xia Jin, xia.jin@uq.edu.au

Packages

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import (MultipleLocator)
import pandas as pd
import numpy as np
import pymc as pm
import arviz as az
import pytensor
import pytensor.tensor as pt
from numpy.ma import masked_values
import pickle
import os
import sys
import time

# C4 biochemical photosynthesis model

The code for simulating C4 biochemical photosynthesis was adapted from Wu et al., (2024) (https://doi.org/10.1111/nph.19537)

Note: ε is represented by alpha_PSII in the code

In [ ]:
# Calculate Cm by starting with Ci with 10 auto-iterations, Cm is used in AC1 calculation.
def tscanCM(np_Cm, Ci, VcmaxT, VpmaxT, RDT, GMT, tempC):
    Ac1 = fAc1((Ci,np_Cm), VcmaxT, VpmaxT, RDT, tempC)
    np_Cm = Ci - Ac1/GMT
    return np_Cm

def fCm(Ci, VcmaxT, VpmaxT, iterations, RDT, tempC):
    GM25=1.2
    GMT=GM25*pt.exp(40600*(tempC+273-298.15)/(298.15*8.314*(tempC+273)))
    np_Cm = Ci

    outInit = np_Cm
    results, updates = pytensor.scan(fn=tscanCM,
                                    outputs_info=outInit,
                                    non_sequences=[Ci, VcmaxT, VpmaxT, RDT, GMT, tempC],
                                    n_steps = iterations)
    return results[-1]


def fI2(PARi, alpha_PSII):
    return PARi*alpha_PSII

def fJt(I2, JmaxT, theta):
    return (I2+JmaxT-np.sqrt((I2+JmaxT)**2-4*theta*JmaxT*I2))/(2*theta)

def fa(Ci,GBST,GMT,RDT,x1,x10,x12,x13,x3,x8,x4,RMT,x5,x6,x7):
    return Ci*GBST - (1/GMT)*RDT*GBST + (1/GMT)*GBST*x1 + GBST*x1*x10 + RDT*GBST*x12 + GBST*x13 + GBST*x3 - RMT*x8 + Ci*x4*x8 - (1/GMT)*RDT*x4*x8 + (1/GMT)*x1*x4*x8 + x5*x8 - RDT*x6*x8 + x1*x6*x8 - x7*x8

def fd(GBST,GMT,x12,x4,x8,x6):
    return (1/GMT)*GBST - GBST*x12 + (1/GMT)*x4*x8 + x6*x8

def fb(GMT,GBST,x4,x8,x12,x6,RMT,RDT,x1,Ci,x13,x3,x5,x7,x11):
    return ((-1/GMT)*GBST - (1/GMT)*x4*x8 + GBST*x12 - x6*x8)*(-RMT*RDT*x8 + RMT*x1*x8 + Ci*RDT*GBST + Ci*RDT*x4*x8 - Ci*GBST*x1 - Ci*x1*x4*x8 + RDT*GBST*x13 + RDT*GBST*x3 + RDT*x5*x8 - RDT*x7*x8 + GBST*x1*x11 - x1*x5*x8 + x1*x7*x8)


def fAc1(data, VcmaxT, VpmaxT, RDT, tempC):
    (Ci, Cm) = data
    GM25=1.2;ALPHA=0.1;GBST=0.003;OM=210000;PHI=2.0;
    GMT=GM25*pt.exp(40600*(tempC+273-298.15)/(298.15*8.314*(tempC+273)))
    KPT=75*pt.exp(36300*(tempC+273-298.15)/(298.15*8.314*(tempC+273)))
    KCT=1210*pt.exp(64200*(tempC+273-298.15)/(298.15*8.314*(tempC+273)))
    KOT=292000*pt.exp(10500*(tempC+273-298.15)/(298.15*8.314*(tempC+273)))
    VCMAXVOMAXT=5.513*pt.exp(21265*(tempC+273-298.15)/(298.15*8.314*(tempC+273)))
    GAMMASTART=0.5/(KOT/KCT*VCMAXVOMAXT)
    RMT=0.5*RDT
    x1=VcmaxT;x3=KCT;x4=VpmaxT/(Cm+KPT);x5=0;x6=1;x7=0;x8=1;x10=(GAMMASTART*ALPHA)/0.047/GBST;x11=GAMMASTART*OM;x12=(KCT/KOT)*ALPHA/0.047/GBST;x13=(KCT/KOT)*OM
    A = fa(Ci,GBST,GMT,RDT,x1,x10,x12,x13,x3,x8,x4,RMT,x5,x6,x7)
    C = A
    D = fd(GBST,GMT,x12,x4,x8,x6)
    B = fb(GMT,GBST,x4,x8,x12,x6,RMT,RDT,x1,Ci,x13,x3,x5,x7,x11)
    Ac1 = (-((A**2 - 4*B)**0.5)+C)/(2*D)
    return Ac1

def fAj(data, theta, alpha_PSII, JmaxT, RDT, tempC):
    (PARi, Ci) = data
    GM25=1.2;ALPHA=0.1;GBST=0.003;OM=210000;PHI=2.0;
    
    GMT=GM25*pt.exp(40600*(tempC+273-298.15)/(298.15*8.314*(tempC+273)))
    KCT=1210*pt.exp(64200*(tempC+273-298.15)/(298.15*8.314*(tempC+273)))
    KOT=292000*pt.exp(10500*(tempC+273-298.15)/(298.15*8.314*(tempC+273)))
    VCMAXVOMAXT=5.513*pt.exp(21265*(tempC+273-298.15)/(298.15*8.314*(tempC+273)))
    GAMMASTART=0.5/(KOT/KCT*VCMAXVOMAXT)
    RMT=0.5*RDT
    X=PHI/(3+PHI)
    FCYC=0.25*PHI
    Z=(3-FCYC)/(4*(1-FCYC))
    I2 = fI2(PARi, alpha_PSII)
    Jt = fJt(I2, JmaxT, theta)
    x1=(Z*(1-X)*Jt/3);x3=0;x4=0;x5=(Z*X*Jt/PHI);x6=1;x7=0;x8=1;x10=(GAMMASTART*ALPHA)/0.047/GBST;x11=GAMMASTART*OM;x12=7.0/3.0*GAMMASTART*ALPHA/0.047/GBST;x13=7.0/3.0*GAMMASTART*OM
    A = fa(Ci,GBST,GMT,RDT,x1,x10,x12,x13,x3,x8,x4,RMT,x5,x6,x7)
    C = A
    D = fd(GBST,GMT,x12,x4,x8,x6)
    B = fb(GMT,GBST,x4,x8,x12,x6,RMT,RDT,x1,Ci,x13,x3,x5,x7,x11)
    Aj = (-((A**2 - 4*B)**0.5)+C)/(2*D)
    return Aj

def fA(Aj,Ac1):
    A = pt.minimum(Ac1, Aj)
    return A

# Bayesian

This code is to read individual paired A/Ci-A/I curves and derive biochemical photosynthetic coefficients Vcmax, Vpmax, ε, Jmax and Rd using a Bayesian approach coupled with a C4 biochemical photosynthesis model described above.

Note: ε is represented by alpha_PSII in the code

In [ ]:
# Read in photosynthesis datafile
data = pd.read_csv("Exp_1.csv", header=0) # Here is am example that we read in the data from Exp_1, you will need to identify the data pathway yourself

# Say which pairs of A/Ci-A/I curves we want to fit
index_values = [15,21,23,24] # Here is an example of A/Ci-A/I curves we select to fit

# Define the funciton to calculate the median of the posterior distribution derived for each biochemical photosynthetic coefficient
def median(x):
    return np.median(x)

# Read in the selected pairs of A/Ci-A/I curves
for index_value in index_values:

    filtered_data = data[data['Index'] == index_value]
    obsAnet = filtered_data["A"].values
    PARi = filtered_data["PARi"].values
    Ci = filtered_data["Ci"].values
    tempC = filtered_data["tempC"].values

    # Build Bayesian model
    model_1 = pm.Model()
    with model_1:
        
        # Define prior distributions of biochemical coefficients we want to derive. Prior distributions were defined referred to Wu et al., (2024) (https://doi.org/10.1111/nph.19537)
        alpha_PSII = pm.Uniform('alpha_PSII', lower=0.01, upper=0.5)
        JmaxT = pm.Uniform('JmaxT', lower=0.01, upper=600)
        RdT = pm.Uniform('RdT', lower=0.01, upper=5)
        theta = 0.3
        RDT = RdT
        Aj = fAj((PARi, Ci), theta, alpha_PSII, JmaxT, RDT, tempC)
        VcmaxT = pm.Uniform('VcmaxT', lower=10, upper=300)
        VpmaxT = pm.Uniform('VpmaxT', lower=10, upper=600)

        # Cm is calculated by using Ci with 10 iterations
        iteration = 10
        Cm = fCm(Ci, VcmaxT, VpmaxT, iteration, RDT, tempC)
        Ac1 = fAc1((Ci, Cm), VcmaxT, VpmaxT, RDT, tempC)

        sigma_Anet = pm.Uniform('sigma_Anet', lower=1e-6, upper=10)

        # Define likilyhood
        Anet = pm.Normal('Anet', mu=fA(Aj, Ac1), sigma=sigma_Anet, observed=obsAnet)

    # Sampling, numbers of draws and tune, target_accept are set based on study needs. Here I set 500 draws with 1500 tune as an example.
    # Draw: The number of samples to be drawn from the posterior distribution after the tuning phase.
    # Tune: The number of samples used for tuning the MCMC algorithm, which are discarded and not used for inference.
    # Target_accept: The target acceptance rate for the proposed samples in the MCMC algorithm.

    idata = pm.sample(model=model_1, draws=5000, tune=15000, target_accept=0.9)

    # Extract and save the draw data from four MCMC sampling chains
    posterior_samples = idata.posterior
    posterior_df = posterior_samples.to_dataframe()

    # Indicating which draw data are from which sampling chain. We have four chains running in the MCMC sampling, the chain index starts from 0
    posterior_df.reset_index(inplace=True)
    posterior_df.rename(columns={'chain': 'Chain'}, inplace=True)

    # Save samples, you will need to identifiy the file pathway yourself
    posterior_df.to_csv(f"idata_{index_value}.csv", index=False) 

    # Plot posterior
    plt.figure(figsize=(12, 8))
    az.plot_trace(idata)
    
    for ax in plt.gcf().axes:
        ax.title.set_fontsize(20)
        for label in ax.get_xticklabels()+ax.get_yticklabels():
            label.set_fontsize(18)

    plt.tight_layout()
    
    # Save plots of posterior distributions, you will need to identifiy the file pathway yourself
    plt.savefig(f"posterior_{index_value}.png", dpi=300)
    plt.close()

    # Save the summary of estiamted posterior distributions, you will need to identifiy the file pathway yourself
    summary_df = az.summary(idata, stat_funcs={'median': np.median})
    summary_df['Treat_index'] = index_value
    summary_df['Coefficient'] = ['alpha_PSII', 'Jmax', 'Rd', 'Vcmax', 'Vpmax', 'sigma_Anet']
    summary_df.to_csv(f"{index_value}.csv", index=False)